In [1]:
using Pkg
#Pkg.add("DifferentialEquations")
#Pkg.add("Calculus")
Pkg.add("Plots")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Libmount_jll ───────────────── v2.35.0+0
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.2+4
   Installed GR_jll ─────────────────────── v0.64.0+0
   Installed JpegTurbo_jll ──────────────── v2.1.2+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed RelocatableFolders ─────────── v0.1.3
   Installed Measures ───────────────────── v0.3.1
   Installed Grisu ──────────────────────── v1.0.2
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed EarCut_jll ─────────────────── v2.2.3+0
   Installed Formatting ─────────────────── v0.4.2
   Installed Contour ────────────────────── v0.5.7
   Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
   Installed Xorg_libpthread_stubs_jll ──── v0.1.0+3
   Installed Cairo_jll ──────────────────── v1.16.1+1
   Installed 

In [2]:
using Calculus
using Plots

CobbDouglas(K, AL, alpha, beta = 1-alpha) = K^alpha * AL^beta

# Production function to be used in model
Production(K, A, L) = CobbDouglas(K, A*L, 0.7)
    
# Capital in (t+1) as function of Capital in (t), Investment in (t)
# and depreciation rate delta
CapitalChange(K,Inv, delta) = (1-delta)*K + Inv

# Investment (equal to savings) as function of Production in (t) and savings rate s
Investment(Y, s) = s * Y

# Labour in (t+1) as function of Labour in (t) and growth rate of population n
LabourChange(L, n) = (1+n)*L

# Technology in (t+1) as function of Technology in (t) and growth rate of technology g
TechnologyChange(A, g) = (1+g)*A

function ExtractParameters(parameters_dict)
    # Takes a Dictionary and creates variables with names corresponding to its key names
    # and values corresponding to its values
           expr = quote end
    for (k, v) in parameters_dict
               push!(expr.args, :($(Symbol(k)) = $v))
           end
           eval(expr)
           return
       end

function SolveFirmsProblem(ProductionFunction, K, A, L)
    production_function_gradient = Calculus.gradient(x -> ProductionFunction(x[1], x[2], x[3]), [K, A, L])
    W = production_function_gradient[3]  # W = dF/dL
    Rk = production_function_gradient[1] # Rk = dF/dK
    W, Rk
end

mutable struct model_parameters
s::Float64
n::Float64
g::Float64
delta::Float64
end

parameters = model_parameters(0.04, 0.02, 0.01, 0.01)

function Solow(K0, A0, L0, parameters, T)
    # In t = 0:
    Y0 = Production(K0, A0, L0)
    Inv0 = Investment(Y0, parameters.s)
    C0 = Y0 - Inv0 # Goods market clearing condition
    W0, Rk0 = SolveFirmsProblem(Production, K0, A0, L0)
    
    # Initializing sequences of allocations and prices for times t=1..T
    for x = [:C, :Y, :K, :Inv, :Rk, :W, :A, :L]
        @eval $x = zeros(T)
    end
    
    t = 1
    K[t] = CapitalChange(K0, Inv0, parameters.delta)
    A[t] = TechnologyChange(A0, parameters.g)
    L[t] = LabourChange(L0, parameters.n)
    
    Y[t] = Production(K[t], A[t], L[t])
    Inv[t] = Investment(Y[t], parameters.s)
    C[t] = Y[t] - Inv[t] # Goods market clearing condition
    W[t], Rk[t] = SolveFirmsProblem(Production, K[t], A[t], L[t])
    
    for t = 2:T
        K[t] = CapitalChange(K[t-1], Inv[t-1], parameters.delta)
        A[t] = TechnologyChange(A[t-1], parameters.g)
        L[t] = LabourChange(L[t-1], parameters.n)
        
        Y[t] = Production(K[t], A[t], L[t])
        Inv[t] = Investment(Y[t], parameters.s)
        C[t] = Y[t] - Inv[t] # Goods market clearing condition
        W[t], Rk[t] = SolveFirmsProblem(Production, K[t], A[t], L[t])
    end
    
    C, Y, K, Inv, Rk, W, A, L
end

K0 = 100
A0 = 1.1
L0 = 300
T = 100

C, Y, K, Inv, Rk, W, A, L = Solow(K0, A0, L0, parameters, T)

plot(C, color="blue")

LoadError: MethodError: no method matching Solow(::Int64, ::Float64, ::Int64, ::model_parameters, ::Int64)
[0mClosest candidates are:
[0m  Solow(::Any, ::Any, ::Any, ::Any, ::Any, [91m::Any[39m, [91m::Any[39m, [91m::Any[39m) at In[2]:38